<a href="https://colab.research.google.com/github/sokoly35/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [7]:
cd dw_matrix

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y= df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [10]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [11]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)


(-57.31783843165656, 4.181246596160967)

In [12]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': 
    return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
      key = item['key'].lower().strip()
      value = item['value'][0].lower().strip()
      output_dict[key] = value
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [14]:
keys = set()


df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [15]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan) 

In [16]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_eye protection type', 'feat_weather-resistant',
       'feat_frame style', 'feat_disclosures', 'feat_packageweight',
       'feat_frame depth', 'feat_leg opening:', 'feat_sub style',
       'feat_fabric care', 'feat_motion control'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [18]:
{k:v for k, v in keys_stat.items() if v> 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


for key in keys:
  df[get_name_feat(key)+ '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [20]:
df['brand'] = df['brand'].map(lambda x : str(x).lower())
df [ df.brand != df.feat_brand].shape


(9434, 531)

In [53]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_clothing category',
 'feat_recommended location',
 'feat_catalog',
 'feat_multi pack indicator',
 'feat_location - country',
 'feat_certifications and listings',
 'feat_shoe category',
 'feat_location - city/state',
 'feat_fabrication',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_manufacturer part number_cat',
 'feat_sport_cat',
 'feat_style _cat',
 'feat_style_cat',
 'feat_sleeve length_cat',
 'feat_manufacturer sku_cat',
 'feat_lens material_cat',
 'feat_age segment_cat',
 'feat_seller inventory number_cat',
 'feat_inv_cat',
 'feat_vehicle type_cat',
 'feat_authenticity_cat',
 'feat_assembled product weight_cat',
 'feat_is dark sky-compliant_cat',
 'feat_lined_cat',
 'feat_item color_cat',
 'feat_era_cat',
 'feat_special features_cat',
 'feat_number of wheels_cat',
 'feat_fits models_cat',
 'feat_adidas_cat',
 'feat_product id_cat',
 'feat_mpn#_cat',
 'feat_diamond clarity_cat',
 'feat_fits brands_cat',
 '

In [0]:
feats = ['brand_cat', 'feat_brand_cat',  'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat' ]
# feats += feats_cat
# feats = list(set(feats))

In [58]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.321666299681056, 4.216769382133619)

In [59]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
 
run_model(feats, model)

(-57.07921879770022, 4.277135120599479)

In [63]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)


perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2608 ± 0.0111,brand_cat
0.1046 ± 0.0083,feat_material_cat
0.0455 ± 0.0046,feat_gender_cat
0.0216 ± 0.0012,feat_brand_cat
0.0070 ± 0.0013,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [31]:
 perm

PermutationImportance(cv='prefit',
                      estimator=RandomForestRegressor(bootstrap=True,
                                                      ccp_alpha=0.0,
                                                      criterion='mse',
                                                      max_depth=5,
                                                      max_features='auto',
                                                      max_leaf_nodes=None,
                                                      max_samples=None,
                                                      min_impurity_decrease=0.0,
                                                      min_impurity_split=None,
                                                      min_samples_leaf=1,
                                                      min_samples_split=2,
                                                      min_weight_fraction_leaf=0.0,
                                                      n_estimators=100,
 

In [39]:
df['brand'].value_counts(normalize=True)

nike                   0.097210
puma                   0.033315
ralph lauren           0.028775
vans                   0.021116
new balance            0.020295
                         ...   
deadstocksunglasses    0.000055
lloyd                  0.000055
and 1                  0.000055
character              0.000055
es                     0.000055
Name: brand, Length: 1732, dtype: float64

In [42]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [50]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'manufacturer part number': '677243-600-13', 'gender': 'men', 'shoe category': "men's shoes", 'brand': 'nike', 'age group': "men's", 'color': 'bright crimson/court purple-total orange'},
       {'style': 'cleats', 'condition': 'new with box', 'shade': 'white-total crimson'},
       {'season': 'all-season', 'shoe size': '11', 'color': 'pink', 'model': '599264 600', 'casual & dress shoe style': 'basketball', 'gender': 'men', 'brand': 'nike', 'age group': 'adult', 'shoe width': 'medium (d, m)', 'material': 'synthetic', 'size': '11', 'shoe category': 'mens shoes', 'manufacturer part number': '599264 600'},
       {'style': 'cushion no-show socks', 'material': 'nylon', 'country/region of manufacture': 'thailand', 'condition': 'new with tags'},
       {'material': 'synthetic', 'gender': 'men', 'size': '9', 'shoe category': "men's shoes", 'color': 'white/anthracite/pure platinum/wolf grey', 'model': 'nikm-flextrainer6w-whiteant455.9m', 'manufacturer part number': 'nikm-flextrainer6w-w

In [51]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
mens                   4
infant                 4
toddler                4
boys'                  3
women ,�� unisex       2
youth                  2
women                  2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64